<a href="https://colab.research.google.com/github/avantika-2001/Stock-Market-Analysis-and-Future-Prediction/blob/main/StockMarketAnalysis.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
!pip install yfinance

In [ ]:
pip show yfinance

In [ ]:
#Importing required libraries
import yfinance as yf
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
from pandas_datareader import data as web

In [ ]:
df_Apple = yf.download(["AAPL"],start='2018-01-01',end='2023-01-01')
df_Apple.head(10)

In [ ]:
df_Apple.describe()

In [ ]:
df_Apple['Adj Close'].plot(legend=True,figsize=(10,4))

In [ ]:
df_Apple['Volume'].plot(legend=True,figsize=(10,4))

In [ ]:
ma_day = [50,100,200]

for ma in ma_day:
    column_name = "MA for %s days" %(str(ma))
    df_Apple[column_name]= df_Apple['Adj Close'].rolling(window=ma).mean()

In [ ]:
df_Apple[['Adj Close','MA for 50 days','MA for 100 days','MA for 200 days']].plot(figsize=(10,4))

In [ ]:
# Daily return percentage
df_Apple['Daily Return'] = df_Apple['Adj Close'].pct_change()
df_Apple['Daily Return'].plot(figsize=(12,4),legend=True,linestyle='--',marker='o')

In [ ]:
df_Apple['Daily Return'].hist()

In [ ]:
closing_df = yf.download(['AAPL','GOOG','MSFT','AMZN'],
start='2018-01-01',
end='2023-01-01')['Adj Close']

In [ ]:
closing_df.head()

In [ ]:
# Daily return analysis
tech_rets = closing_df.pct_change()

In [ ]:
#Correlation btw different stocks
correlation=tech_rets.corr()
correlation.style.background_gradient(cmap = 'hot')  

**RISK** **ANALYSIS**

In [ ]:
rets = tech_rets.dropna()
days = 365
dt = 1/days
mu = rets.mean()['GOOG']
sigma = rets.std()['GOOG']

In [ ]:
def stock_monte_carlo(start_price,days,mu,sigma):    
    price = np.zeros(days)
    price[0] = start_price
    # Shock and Drift
    shock = np.zeros(days)
    drift = np.zeros(days)
    for x in range(1,days):
        shock[x] = np.random.normal(loc=mu * dt, scale=sigma * np.sqrt(dt))
        drift[x] = mu * dt
        price[x] = price[x-1] + (price[x-1] * (drift[x] + shock[x]))        
    return price

In [ ]:
# Get start price from GOOG.head()
start_price = 540.00

for run in range(100):
    plt.plot(stock_monte_carlo(start_price,days,mu,sigma))
plt.xlabel("Days")
plt.ylabel("Price")  
plt.title('Monte Carlo Analysis for Google')

In [ ]:
runs = 10000
# Create an empty matrix to hold the end price data
simulations = np.zeros(runs)
for run in range(runs):    
        simulations[run] = stock_monte_carlo(start_price,days,mu,sigma)[days-1];

In [ ]:
q = np.percentile(simulations, 1)
plt.hist(simulations,bins=200)

plt.figtext(0.6, 0.8, s="Start price: $%.2f" %start_price)
# Mean ending price
plt.figtext(0.6, 0.7, "Mean final price: $%.2f" % simulations.mean())

# Variance of the price (within 99% confidence interval)
plt.figtext(0.6, 0.6, "VaR(0.99): $%.2f" % (start_price - q))

# Display 1% quantile
plt.figtext(0.15, 0.6, "q(0.99): $%.2f" % q)

# Plot a line at the 1% quantile result
plt.axvline(x=q, linewidth=4, color='r')
plt.title(u"Final price distribution for Google Stock after %s days" % days, weight='bold');